In [1]:
import sys
sys.path.insert(1, '../unseen')
import pdb

from dask.distributed import Client, LocalCluster
import xclim.analog

import fileio
import bootstrap

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 28.00 GiB
Comm: tcp://127.0.0.1:36609,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 28.00 GiB
Comm: tcp://127.0.0.1:38571,Total threads: 2
Dashboard: http://127.0.0.1:43143/status,Memory: 7.00 GiB
Nanny: tcp://127.0.0.1:39293,


## Read data

In [4]:
cafe_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_awap-additive-correction_19900501-19931101_A-DEC-sum_cafe-grid-TAS-POINT.zarr.zip'
ds_cafe = fileio.open_file(cafe_file)
ds_cafe          

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 8, lead_time: 11)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (lead_time, init_date) object dask.array<chunksize=(11, 8), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble) float64 dask.array<chunksize=(8, 11, 96), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 14:12:35 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [5]:
awap_file = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'
ds_awap = fileio.open_file(awap_file)   #, sel={'time': slice('1980-01-01', '2000-12-31')})
ds_awap

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) object 1900-12-31 00:00:00 ... 2020-12-31 00:00:00
Data variables:
    pr       (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 13:54:07 2021: /g/data/e14/dbi599/miniconda3/envs/un...

## K-S test

In [6]:
fcst_stacked = ds_cafe.stack({'sample': ['ensemble', 'init_date', 'lead_time']})

In [7]:
fcst_stacked['pr']

<xarray.DataArray 'pr' (sample: 8448)>
dask.array<reshape, shape=(8448,), dtype=float64, chunksize=(8448,), chunktype=numpy.ndarray>
Coordinates:
    time       (sample) object dask.array<chunksize=(8448,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 1 1 1 1 1 1 1 1 1 1 ... 96 96 96 96 96 96 96 96 96
  - init_date  (sample) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  - lead_time  (sample) int64 0 1 2 3 4 5 6 7 8 9 10 ... 0 1 2 3 4 5 6 7 8 9 10
Attributes:
    bias_correction_method:  additive

In [8]:
awap_faux_stacked = ds_awap.rename(time='sample')
#awap_faux_stacked['sample'] = np.arange(awap_faux_stacked['sample'].size)
awap_faux_stacked = awap_faux_stacked.chunk({'sample': -1})
awap_faux_stacked['pr']

<xarray.DataArray 'pr' (sample: 121)>
dask.array<rechunk-merge, shape=(121,), dtype=float64, chunksize=(121,), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object 1900-12-31 00:00:00 ... 2020-12-31 00:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm d-1
    valid_range:                   [-1.0, 100000.0]

In [9]:
ks_obs = xclim.analog.spatial_analogs(target=awap_faux_stacked,
                                      candidates=fcst_stacked,
                                      dist_dim='sample',
                                      method='kolmogorov_smirnov')

In [10]:
ks_obs.values

array(0.10890152)

For a 2D KS-test I think you just include two variables in the Dataset instead of just one. For instance, at the moment `data_vars` only has one variable, but you could add another.

In [11]:
fcst_stacked.data_vars

Data variables:
    pr       (sample) float64 dask.array<chunksize=(8448,), meta=np.ndarray>

# Resampling (bootstrapping)

Need to test random sub-samples from the forecast data (with same length as the observations) against the entire forecast sample to get the distribution of values you might expect if the observations come from the same population as the forecast. 

In [12]:
n_samples = len(awap_faux_stacked['sample'])
n_samples

121

In [13]:
samples = {'sample': (n_samples, 1)}

### Single random sample

In [14]:
fcst_random_sample = bootstrap.random_resample(fcst_stacked, samples=samples)
fcst_random_sample

<xarray.Dataset>
Dimensions:    (sample: 121)
Coordinates:
    time       (sample) object dask.array<chunksize=(121,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 7 1 34 90 16 63 60 63 6 ... 7 91 90 77 66 76 49 19
  - init_date  (sample) object 1992-12-31 00:00:00 ... 1993-12-31 00:00:00
  - lead_time  (sample) int64 4 1 3 6 7 0 6 8 0 8 1 5 ... 5 7 9 7 3 6 4 2 4 3 10
Data variables:
    pr         (sample) float64 dask.array<chunksize=(121,), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 14:12:35 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [15]:
ks_random = xclim.analog.spatial_analogs(target=fcst_random_sample,
                                         candidates=fcst_stacked,
                                         dist_dim='sample',
                                         method='kolmogorov_smirnov')

In [16]:
ks_random

<xarray.DataArray 'dissimilarity' ()>
dask.array<transpose, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
Attributes:
    long_name:  Dissimilarity between target and candidates, using metric kol...
    indices:    pr
    metric:     kolmogorov_smirnov

In [17]:
ks_random.values

array(0.06013258)

You get the same result no matter which array is the target and which is the candidate.

### n random samples

In [18]:
n_repeats = 50
function = xclim.analog.spatial_analogs
function_kwargs = {'candidates': fcst_stacked,
                   'dist_dim': 'sample',
                   'method': 'kolmogorov_smirnov'}

dist = bootstrap.n_random_resamples(fcst_stacked,
                                    samples=samples,
                                    n_repeats=n_repeats,
                                    function=function,
                                    function_kwargs=function_kwargs,
                                    bundle_args=False,
                                    with_dask=True)

In [19]:
dist

<xarray.DataArray 'dissimilarity' (k: 50)>
array([0.06837552, 0.08134254, 0.0462401 , 0.07318569, 0.06663223,
       0.08479683, 0.07848011, 0.12346117, 0.09476154, 0.09722581,
       0.09499828, 0.07804967, 0.05447228, 0.07940556, 0.07682292,
       0.07524105, 0.0450241 , 0.11463714, 0.04173123, 0.09142562,
       0.07025869, 0.05686123, 0.07399277, 0.07419723, 0.08959625,
       0.04144069, 0.12907843, 0.08983299, 0.05213714, 0.09237259,
       0.06121944, 0.05544077, 0.09416968, 0.09112431, 0.04522856,
       0.04303332, 0.08517347, 0.08840177, 0.05968061, 0.0591318 ,
       0.13778409, 0.07919034, 0.06659995, 0.07503659, 0.10455406,
       0.09626808, 0.06551309, 0.09708592, 0.09046789, 0.06778366])
Dimensions without coordinates: k
Attributes:
    long_name:  Dissimilarity between target and candidates, using metric kol...
    indices:    pr
    metric:     kolmogorov_smirnov

In [20]:
dist.values

array([0.06837552, 0.08134254, 0.0462401 , 0.07318569, 0.06663223,
       0.08479683, 0.07848011, 0.12346117, 0.09476154, 0.09722581,
       0.09499828, 0.07804967, 0.05447228, 0.07940556, 0.07682292,
       0.07524105, 0.0450241 , 0.11463714, 0.04173123, 0.09142562,
       0.07025869, 0.05686123, 0.07399277, 0.07419723, 0.08959625,
       0.04144069, 0.12907843, 0.08983299, 0.05213714, 0.09237259,
       0.06121944, 0.05544077, 0.09416968, 0.09112431, 0.04522856,
       0.04303332, 0.08517347, 0.08840177, 0.05968061, 0.0591318 ,
       0.13778409, 0.07919034, 0.06659995, 0.07503659, 0.10455406,
       0.09626808, 0.06551309, 0.09708592, 0.09046789, 0.06778366])